In [144]:
import threading
import uuid
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

In [145]:
successful_transactions = 0
failed_transactions = 0
successful_rebalance_count = 0
failed_rebalance_count = 0



In [146]:
class Rack:
    def __init__(self, capacity, quantity, rack_type):
        self.id = uuid.uuid4()
        self.capacity = int(capacity)
        self.quantity = int(quantity)
        self.rack_type = rack_type
        self.lock = threading.RLock()  # Using RLock instead of a normal Lock
        if self.quantity > self.capacity:
            raise ValueError("Rack capacity exceeded.")
        if self.quantity < 0:
            raise ValueError("Rack negative quantity.")

    def is_full(self):
        return self.quantity >= self.capacity

    def remaining_capacity(self):
        return self.capacity - self.quantity

    def __repr__(self):
        c_val = self.capacity / 1_000_000
        q_val = self.quantity / 1_000_000
        r_val = self.remaining_capacity() / 1_000_000
        return (f"Rack(ID: {self.id}, Type: {self.rack_type}, "
                f"Capacity: {c_val}g, Quantity: {q_val}g, Remaining: {r_val}g, "
                f"Full: {self.is_full()})")


In [147]:
warehouse_capacity = 100_000_000
warehouse_quantity = 80_000_000
warehouse = Rack(warehouse_capacity, warehouse_quantity, "WAREHOUSE")
warehouse_lock = threading.RLock()

In [148]:
out_racks = {
    Rack(10_000_000, 5_000_000, "OUT"),
    Rack(15_000_000, 8_000_000, "OUT"),
    Rack(20_000_000, 18_000_000, "OUT"),
    Rack(25_000_000, 10_000_000, "OUT"),
    Rack(30_000_000, 28_000_000, "OUT")
}

in_racks = {
    Rack(20_000_000, 10_000_000, "IN"),
    Rack(25_000_000, 20_000_000, "IN"),
    Rack(30_000_000, 15_000_000, "IN"),
    Rack(35_000_000, 30_000_000, "IN"),
    Rack(40_000_000, 18_000_000, "IN")
}

In [149]:

locked_racks = set()
unlocked_racks = set(out_racks | in_racks)

In [150]:
def rebalance_out_for_gold(rack_obj, needed_amount, warehouse_obj):
    global successful_rebalance_count, failed_rebalance_count
    short = needed_amount - rack_obj.quantity
    locked_warehouse = warehouse_lock.acquire(timeout=1)
    if not locked_warehouse:
        failed_rebalance_count += 1
        return False
    locked_rack = rack_obj.lock.acquire(timeout=1)
    if not locked_rack:
        warehouse_lock.release()
        failed_rebalance_count += 1
        return False
    try:
        if warehouse_obj.quantity < short or (rack_obj.quantity + short) > rack_obj.capacity:
            failed_rebalance_count += 1
            rack_obj.lock.release()
            warehouse_lock.release()
            locked_racks.discard(rack_obj)
            unlocked_racks.add(rack_obj)
            return False
        warehouse_obj.quantity -= short
        rack_obj.quantity += short
        successful_rebalance_count += 1
        rack_obj.lock.release()
        warehouse_lock.release()
        locked_racks.discard(rack_obj)
        unlocked_racks.add(rack_obj)
        return True
    except:
        rack_obj.lock.release()
        warehouse_lock.release()
        locked_racks.discard(rack_obj)
        unlocked_racks.add(rack_obj)
        raise

In [151]:
def rebalance_out_for_capacity(rack_obj, needed_amount, warehouse_obj):
    global successful_rebalance_count, failed_rebalance_count
    space_needed = needed_amount - rack_obj.remaining_capacity()
    if not warehouse_lock.acquire(timeout=1):
        failed_rebalance_count += 1
        return False
    if not rack_obj.lock.acquire(timeout=1):
        warehouse_lock.release()
        failed_rebalance_count += 1
        return False
    try:
        if warehouse_obj.remaining_capacity() < space_needed or rack_obj.quantity < space_needed:
            failed_rebalance_count += 1
            rack_obj.lock.release()
            warehouse_lock.release()
            locked_racks.discard(rack_obj)
            unlocked_racks.add(rack_obj)
            return False
        warehouse_obj.quantity += space_needed
        rack_obj.quantity -= space_needed
        successful_rebalance_count += 1
        rack_obj.lock.release()
        warehouse_lock.release()
        locked_racks.discard(rack_obj)
        unlocked_racks.add(rack_obj)
        return True
    except:
        rack_obj.lock.release()
        warehouse_lock.release()
        locked_racks.discard(rack_obj)
        unlocked_racks.add(rack_obj)
        raise

In [152]:
def select_source_rack(racks_set, need, rack_type, warehouse_obj):
    if rack_type == "OUT":
        cands = []
        for r in unlocked_racks:
            if r.rack_type == "OUT" and r.quantity >= need:
                cands.append(r)
        if cands:
            chosen = cands[0]
            for c2 in cands:
                if c2.remaining_capacity() < chosen.remaining_capacity():
                    chosen = c2
            got_lock = chosen.lock.acquire(timeout=1)
            if got_lock:
                unlocked_racks.remove(chosen)
                locked_racks.add(chosen)
                return chosen
        best_r = None
        max_qty = -1
        for r2 in unlocked_racks:
            if r2.rack_type == "OUT" and r2.quantity < need:
                if r2.quantity > max_qty:
                    best_r = r2
                    max_qty = r2.quantity
        if best_r and best_r.lock.acquire(timeout=1):
            unlocked_racks.remove(best_r)
            locked_racks.add(best_r)
            succ = rebalance_out_for_gold(best_r, need, warehouse_obj)
            if succ:
                return best_r
            best_r.lock.release()
            locked_racks.remove(best_r)
            unlocked_racks.add(best_r)
        return None
    else:
        in_candidates = []
        for r3 in unlocked_racks:
            if r3.rack_type == "IN" and r3.quantity >= need:
                in_candidates.append(r3)
        if in_candidates:
            chosen_in = in_candidates[0]
            for c3 in in_candidates:
                if c3.remaining_capacity() < chosen_in.remaining_capacity():
                    chosen_in = c3
            got_lock_in = chosen_in.lock.acquire(timeout=1)
            if got_lock_in:
                unlocked_racks.remove(chosen_in)
                locked_racks.add(chosen_in)
                return chosen_in
        return None

In [153]:
def select_destination_rack(need, rack_type, warehouse_obj):
    if rack_type == "OUT":
        outs = []
        for r in unlocked_racks:
            if r.rack_type == "OUT" and r.remaining_capacity() >= need:
                outs.append(r)
        if outs:
            best_o = outs[0]
            for c2 in outs:
                if c2.remaining_capacity() > best_o.remaining_capacity():
                    best_o = c2
            got_lock_o = best_o.lock.acquire(timeout=1)
            if got_lock_o:
                unlocked_racks.remove(best_o)
                locked_racks.add(best_o)
                return best_o
        best_reb = None
        high_cap = -1
        for r2 in unlocked_racks:
            if r2.rack_type == "OUT" and r2.remaining_capacity() < need:
                if r2.capacity > high_cap:
                    best_reb = r2
                    high_cap = r2.capacity
        if best_reb and best_reb.lock.acquire(timeout=1):
            unlocked_racks.remove(best_reb)
            locked_racks.add(best_reb)
            ok_cap = rebalance_out_for_capacity(best_reb, need, warehouse_obj)
            if ok_cap:
                return best_reb
            best_reb.lock.release()
            locked_racks.remove(best_reb)
            unlocked_racks.add(best_reb)
        return None
    else:
        ins = []
        for r3 in unlocked_racks:
            if r3.rack_type == "IN" and r3.remaining_capacity() >= need:
                ins.append(r3)
        if ins:
            best_i = ins[0]
            for c4 in ins:
                if c4.remaining_capacity() > best_i.remaining_capacity():
                    best_i = c4
            lock_i = best_i.lock.acquire(timeout=1)
            if lock_i:
                unlocked_racks.remove(best_i)
                locked_racks.add(best_i)
                return best_i
        return None


In [154]:
def buy_gold(amount, out_set, in_set, warehouse_obj):
    global successful_transactions, failed_transactions
    s_rack = select_source_rack(out_set, amount, "OUT", warehouse_obj)
    if not s_rack:
        failed_transactions += 1
        return False, None, None
    d_rack = select_destination_rack(amount, "IN", warehouse_obj)
    if not d_rack:
        failed_transactions += 1
        s_rack.lock.release()
        locked_racks.discard(s_rack)
        unlocked_racks.add(s_rack)
        return False, None, None
    first = s_rack if id(s_rack) < id(d_rack) else d_rack
    second = d_rack if first is s_rack else s_rack
    locked_1 = first.lock.acquire(timeout=1)
    if not locked_1:
        failed_transactions += 1
        return False, None, None
    locked_2 = second.lock.acquire(timeout=1)
    if not locked_2:
        first.lock.release()
        failed_transactions += 1
        return False, None, None
    try:
        if s_rack.quantity < amount or d_rack.quantity + amount > d_rack.capacity:
            failed_transactions += 1
            return False, s_rack, d_rack
        s_rack.quantity -= amount
        d_rack.quantity += amount
        successful_transactions += 1
        return True, s_rack, d_rack
    finally:
        second.lock.release()
        first.lock.release()
        locked_racks.discard(s_rack)
        locked_racks.discard(d_rack)
        unlocked_racks.add(s_rack)
        unlocked_racks.add(d_rack)


In [155]:
def sell_gold(amount, in_set, out_set, warehouse_obj):
    global successful_transactions, failed_transactions
    s_in = select_source_rack(in_set, amount, "IN", warehouse_obj)
    if not s_in:
        failed_transactions += 1
        return False, None, None
    d_out = select_destination_rack(amount, "OUT", warehouse_obj)
    if not d_out:
        failed_transactions += 1
        s_in.lock.release()
        locked_racks.discard(s_in)
        unlocked_racks.add(s_in)
        return False, None, None
    f_lock = s_in if id(s_in) < id(d_out) else d_out
    s_lock = d_out if f_lock is s_in else s_in
    l1 = f_lock.lock.acquire(timeout=1)
    if not l1:
        failed_transactions += 1
        return False, None, None
    l2 = s_lock.lock.acquire(timeout=1)
    if not l2:
        f_lock.lock.release()
        failed_transactions += 1
        return False, None, None
    try:
        if s_in.quantity < amount or d_out.quantity + amount > d_out.capacity:
            failed_transactions += 1
            return False, s_in, d_out
        s_in.quantity -= amount
        d_out.quantity += amount
        successful_transactions += 1
        return True, s_in, d_out
    finally:
        s_lock.lock.release()
        f_lock.lock.release()
        locked_racks.discard(s_in)
        locked_racks.discard(d_out)
        unlocked_racks.add(s_in)
        unlocked_racks.add(d_out)


In [156]:
def generate_transaction_list(buy_r, sell_r, total_ops, bmin, bmax, smin, smax):
    transactions = []
    total_mix = buy_r + sell_r
    if total_mix == 0 or total_ops <= 0:
        return transactions
    buy_count = round((buy_r / total_mix) * total_ops)
    sell_count = total_ops - buy_count
    i = 0
    while i < buy_count:
        g_val = round(random.uniform(bmin, bmax), 4)
        micro = int(g_val * 1_000_000)
        transactions.append(("BUY", micro))
        i += 1
    j = 0
    while j < sell_count:
        g_val2 = round(random.uniform(smin, smax), 4)
        micro2 = int(g_val2 * 1_000_000)
        transactions.append(("SELL", micro2))
        j += 1
    random.shuffle(transactions)
    return transactions


In [157]:
def run_simulation(buy_r, sell_r, total_ops, bmin, bmax, smin, smax):
    global successful_transactions, failed_transactions
    print("\n=== INITIAL STATE ===")
    print("\nIN Racks:")
    for r_in in in_racks:
        print(r_in)
    print("\nOUT Racks:")
    for r_out in out_racks:
        print(r_out)
    print(f"\nWarehouse: {warehouse}")
    tx_list = generate_transaction_list(buy_r, sell_r, total_ops, bmin, bmax, smin, smax)
    if not tx_list:
        print("\nNo valid transactions.")
        return
    with ThreadPoolExecutor(max_workers=5) as thread_pool:
        f_map = {}
        for tx_type, mic_val in tx_list:
            g_val = mic_val / 1_000_000
            print(f"\nGenerated Transaction: {tx_type} {g_val:.6f}g ({mic_val} µg)")
            if tx_type == "BUY":
                future = thread_pool.submit(buy_gold, mic_val, out_racks, in_racks, warehouse)
            else:
                future = thread_pool.submit(sell_gold, mic_val, in_racks, out_racks, warehouse)
            f_map[future] = (tx_type, g_val, mic_val)
        for ft_done in as_completed(f_map):
            kind, grams, microz = f_map[ft_done]
            try:
                success, s_r, d_r = ft_done.result()
                if success:
                    print(f"Transaction {kind} {grams:.6f}g SUCCESS")
                else:
                    print(f"Transaction {kind} {grams:.6f}g FAILED")
            except Exception as exc:
                print(f"Transaction {kind} {grams:.6f}g EXCEPTION: {exc}")
    print("\n=== FINAL STATE ===")
    print("\nIN Racks:")
    for rin in in_racks:
        print(rin)
    print("\nOUT Racks:")
    for rout in out_racks:
        print(rout)
    print(f"\nWarehouse: {warehouse}")
    print(f"\nTotal Successful Transactions: {successful_transactions}")
    print(f"Total Failed Transactions: {failed_transactions}")
    print(f"Total Successful Rebalances: {successful_rebalance_count}")
    print(f"Total Failed Rebalances: {failed_rebalance_count}")

In [158]:
run_simulation(
    buy_r=2,        # ratio of BUY transactions
    sell_r=1,       # ratio of SELL transactions
    total_ops=10,   # total number of transactions
    bmin=0.0001,    # minimum BUY amount in grams
    bmax=40.0,      # maximum BUY amount in grams
    smin=0.001,     # minimum SELL amount in grams
    smax=40.0       # maximum SELL amount in grams
)


=== INITIAL STATE ===

IN Racks:
Rack(ID: 80cc1142-9aa1-48be-a804-8ee15830a921, Type: IN, Capacity: 35.0g, Quantity: 30.0g, Remaining: 5.0g, Full: False)
Rack(ID: f2582a22-1b07-4b89-a0eb-735abc9d0c5c, Type: IN, Capacity: 20.0g, Quantity: 10.0g, Remaining: 10.0g, Full: False)
Rack(ID: 5f4ac684-7400-4e71-991a-0300a0efb3ef, Type: IN, Capacity: 40.0g, Quantity: 18.0g, Remaining: 22.0g, Full: False)
Rack(ID: c79f1da8-2581-4a1e-afc3-0d7a3abce51b, Type: IN, Capacity: 25.0g, Quantity: 20.0g, Remaining: 5.0g, Full: False)
Rack(ID: 36499624-6bf3-4608-aec0-7a5ce5b50a81, Type: IN, Capacity: 30.0g, Quantity: 15.0g, Remaining: 15.0g, Full: False)

OUT Racks:
Rack(ID: a6a3287c-4754-4449-bbaf-d1db8e9b769a, Type: OUT, Capacity: 20.0g, Quantity: 18.0g, Remaining: 2.0g, Full: False)
Rack(ID: b63e472c-18c0-4d8b-ab78-5b0cfa9af7ad, Type: OUT, Capacity: 30.0g, Quantity: 28.0g, Remaining: 2.0g, Full: False)
Rack(ID: a41917c1-3488-4677-b2dc-7e6b8e7a8eaf, Type: OUT, Capacity: 25.0g, Quantity: 10.0g, Remaining: